In [ ]:
from googleapiclient.discovery import build
import pandas as pd
import seaborn as sns

In [ ]:
api_key = 'your_key_here'


channel_ids = [ 
                
                'UCLLw7jmFsvfIVaUFsLs8mlQ', # Luke Barousse
                'UCiT9RITQ9PW6BhXK0y2jaeg', # Ken Jee
                'UCBwmMxybNva6P_5VmxjzwqA', # Apna College
                'UC2UXDak6o7rBm23k3Vv5dww'  # Tina Huang
              ]
# get yt service based on that we raise request to API

# get service 
youtube = build("youtube", "v3", developerKey=api_key )

#### extract channels details
### Function to get channel statistic


In [ ]:
def get_channel_stats(youtube,channel_ids):
    all_data=[]
    request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=','.join(channel_ids))
    responce = request.execute()
    print("Returned channel IDs:", [item['id'] for item in responce['items']])

    for i in range(len(responce['items'])):
        try:
            data = dict(Channel_name = responce['items'][i]['snippet']['title'],
                        Subscribers = responce['items'][i]['statistics']['subscriberCount'],
                        Views = responce['items'][i]['statistics']['viewCount'],
                        Total_videos = responce['items'][i]['statistics']['videoCount'],
                        playlist_id = responce['items'][i]['contentDetails']['relatedPlaylists']['uploads']
                        )
            all_data.append(data)
        except KeyError as e:
            print(f"Error fetching data for one channel: {e}")
            continue
    return all_data

In [ ]:
channel_statistics = get_channel_stats(youtube,channel_ids)

### Load data into Dataframe for Visuallization

In [ ]:
channel_data = pd.DataFrame(channel_statistics)
# channel_data

In [ ]:
# Convert [ Object --> Int]
channel_data['Subscribers'] = pd.to_numeric(channel_data['Subscribers'])
channel_data['Views'] = pd.to_numeric(channel_data['Views'])
channel_data['Total_videos'] = pd.to_numeric(channel_data['Total_videos'])
channel_data.dtypes

In [ ]:
# increase size
sns.set(rc={'figure.figsize':(10,8)})
ax =sns.barplot(x='Channel_name',y='Subscribers',data=channel_data)

In [ ]:
ax =sns.barplot(x='Channel_name',y='Views',data=channel_data)

In [ ]:
ax =sns.barplot(x='Channel_name',y='Total_videos',data=channel_data)

In [ ]:
# write function to extract playlist_id from dataframe n load
playlist_id = channel_data.loc[channel_data['Channel_name']=='Apna College','playlist_id'].iloc[0]
playlist_id

In [ ]:
# Write function to fetch video id's
# then write another function to ectract video info

def get_video_ids(youtube,playlist_id):
    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50
                )
    responce= request.execute()
    return responce

In [ ]:
def get_video_ids(youtube,playlist_id):
    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50
                )
    responce= request.execute()

    video_ids=[]

    for i in range(len(responce['items'])):
        video_ids.append(responce['items'][i]['contentDetails']['videoId'])
    
    next_page_token =responce.get('nextPageToken')
    more_pages =True

    while more_pages:
        if next_page_token is None:
            more_pages =False
        else:
            request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50,
                pageToken = next_page_token)
            responce= request.execute()

            for i in range(len(responce['items'])):
                video_ids.append(responce['items'][i]['contentDetails']['videoId'])

            next_page_token = responce.get('nextPageToken')

    # return responce
    return video_ids
    # return len(video_ids)

In [ ]:
video_ids = get_video_ids(youtube,playlist_id)

In [ ]:
def get_video_details(youtube, video_ids):
    if not isinstance(video_ids, (list, tuple)):
        raise TypeError("video_ids must be a list or tuple of video ID strings.")
    
    all_video_stats=[]

    for i in range(0,len(video_ids),50):
        request = youtube.videos().list(
                    part='snippet,statistics',
                    id= ','.join(video_ids[i:i+50]))
        responce = request.execute()

        for video in responce['items']:
            video_stats = dict(Title= video['snippet']['title'],
                               published_date = video['snippet']['publishedAt'],
                            #    Views= video['statistics']['viewCount'],
                               Likes= video['statistics']['likeCount'],
                               favourite= video['statistics']['favoriteCount'],
                               Comments= video['statistics']['commentCount'],
                               )
            all_video_stats.append(video_stats)

    # return len(all_video_stats)
    return all_video_stats


In [ ]:
video_details = get_video_details(youtube, video_ids)

In [ ]:
video_data = pd.DataFrame(video_details)
# video_data

In [ ]:
video_data['published_date']= pd.to_datetime(video_data['published_date']).dt.date
# video_data['Views']=pd.to_numeric(video_data['Views'])
video_data['Likes']=pd.to_numeric(video_data['Likes'])
video_data['favourite']=pd.to_numeric(video_data['favourite'])
video_data['Comments']=pd.to_numeric(video_data['Comments'])
video_data

In [ ]:
ax1 = sns.barplot(x='Views',y='Title',data= top10_videos)

In [ ]:
video_data.to_csv('video_details(Apna College).csv')